Problema simplificado de los ascensores, primera instancia del problema en el que el edificio contiene un único bloque y todos los ascensores presentan las mismas características.

In [1]:
from search import *
import copy

In [2]:
NUM_PLANTAS    = 5
NUM_PERSONAS   = 3
NUM_ASCENSORES = 1
CAP_ASCENSORES = 2
EN_ASCENSOR = -1

BAJAR=0
SUBIR=1
COGER_PERSONA=2
DEJAR_PERSONA=3

In [3]:
personas_pos_ini = [0]
personas_pos_fin = [1]

In [45]:
class NodeState:

    def __init__(self, init_personas):
        self.personas = init_personas
        self.ascensor_pos = 0
        self.ascensor_personas = []
        

In [46]:
init = NodeState(personas_pos_ini)

In [47]:
goal = NodeState(personas_pos_fin)

In [48]:
class AscensoresSimplificado(Problem) :    
   
    # Hecho
    def __init__(self, initial, goal=None):        
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        accs = list()
        #Para cada ascensor
        num_ascensor = 0
        ascensor_pos = state.ascensor_pos
        ascensor_personas = state.ascensor_personas
        personas = state.personas
        
        print("Lista acciones en estado -> Ascensor:"+str(ascensor_pos)+",Posicion_persona:"+str(personas[0])+
             ",Gente en el ascensor: "+str(ascensor_personas))
        
        #si alguien ha llegado a su destino, hay que forzar que salga
        forzar_salida = -1
        for persona in ascensor_personas:
            if ascensor_pos==self.goal.personas[persona]:
                forzar_salida = persona
        if forzar_salida !=-1:
            print("Puedes forzar")
            accs.append((DEJAR_PERSONA,num_ascensor,forzar_salida))
            return accs
        
        #si hay alguien en un ascensor, marca si el ascensor sube o baja
        #maximo del bloque
        if ascensor_pos != NUM_PLANTAS and not(len(ascensor_personas)>0 and self.goal.personas[ascensor_personas[0]]<ascensor_pos):
            accs.append((SUBIR,num_ascensor))
            print("Puedes subir")
        #minimo del bloque
        if ascensor_pos != 0 and not(len(ascensor_personas)>0 and self.goal.personas[ascensor_personas[0]]>ascensor_pos):
            accs.append((BAJAR,num_ascensor))
            print("Puedes bajar")
        for personaInterior in ascensor_personas:
            accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))
            print("Puedes dejar persona "+str(personaInterior))
        if len(ascensor_personas)<CAP_ASCENSORES:
            for i in range(0,len(personas)):
                if personas[i]==ascensor_pos: #la gente en ascensor tiene pos EN_ASCENSOR
                    accs.append((COGER_PERSONA,num_ascensor,i))
                    print("Puedes coger persona "+str(i))
        
        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        ascensor = action[1]
        ascensor_pos = state.ascensor_pos
        ascensor_personas = state.ascensor_personas
        personas = state.personas
        print("Accion ejecutada: "+str(action[0]))
        print("Antes -> Ascensor:"+str(ascensor_pos)+",Posicion_persona:"+str(personas[0]))
        if action[0]==BAJAR:
            nueva_pos_ascensor = 0 #min bloque
            #si hay gente dentro:
            if(len(ascensor_personas)>0):
                nueva_pos_ascensor = max([self.goal.personas[i] for i in ascensor_personas])
            for i in range(0,len(personas)):
                if nueva_pos_ascensor<personas[i] and personas[i]<ascensor_pos:
                    nueva_pos_ascensor = personas[i]
            estado_nuevo.ascensor_pos = nueva_pos_ascensor
            print("Bajar de planta " + str(ascensor_pos)+ " a " +str(nueva_pos_ascensor))
        elif action[0]==SUBIR:
            nueva_pos_ascensor = NUM_PLANTAS #max bloque
            #si hay gente dentro:
            if(len(ascensor_personas)>0):
                nueva_pos_ascensor = min([self.goal.personas[i] for i in ascensor_personas])
            for i in range(0,len(personas)):
                if nueva_pos_ascensor>personas[i] and personas[i]>ascensor_pos:
                    nueva_pos_ascensor = personas[i]
            estado_nuevo.ascensor_pos = nueva_pos_ascensor
            print("Subir de planta " + str(ascensor_pos)+ " a " +str(nueva_pos_ascensor))
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas.append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
            print("Coger persona "+str(action[2])+" en planta "+str(ascensor_pos))
        else:
            estado_nuevo.ascensor_personas.remove(action[2])
            estado_nuevo.personas[action[2]] = ascensor_pos
            print("Dejar persona "+str(action[2])+" en planta "+str(ascensor_pos))
        
        print("Despues -> Ascensor:"+str(estado_nuevo.ascensor_pos)+",Posicion_persona:"+str(estado_nuevo.personas[0])+
             ",Gente en el ascensor: "+str(estado_nuevo.ascensor_personas))
        return estado_nuevo

    # Hecho
    def goal_test(self, state):
        return state.personas == self.goal.personas

    def path_cost(self, c, state1, action, state2):
        return 1

    def value(self, state):
        raise NotImplementedError

In [49]:
problem = AscensoresSimplificado(init, goal)

In [50]:
breadth_first_graph_search(problem).solution()

Lista acciones en estado -> Ascensor:0,Posicion_persona:0,Gente en el ascensor: []
Puedes subir
Puedes coger persona 0
Accion ejecutada: 1
Antes -> Ascensor:0,Posicion_persona:0
Subir de planta 0 a 5
Despues -> Ascensor:5,Posicion_persona:0,Gente en el ascensor: []
Accion ejecutada: 2
Antes -> Ascensor:0,Posicion_persona:0
Coger persona 0 en planta 0
Despues -> Ascensor:0,Posicion_persona:-1,Gente en el ascensor: [0]
Lista acciones en estado -> Ascensor:5,Posicion_persona:0,Gente en el ascensor: []
Puedes bajar
Accion ejecutada: 0
Antes -> Ascensor:5,Posicion_persona:0
Bajar de planta 5 a 0
Despues -> Ascensor:0,Posicion_persona:0,Gente en el ascensor: []
Lista acciones en estado -> Ascensor:0,Posicion_persona:-1,Gente en el ascensor: [0]
Puedes subir
Puedes dejar persona 0
Accion ejecutada: 1
Antes -> Ascensor:0,Posicion_persona:-1
Subir de planta 0 a 1
Despues -> Ascensor:1,Posicion_persona:-1,Gente en el ascensor: [0]
Accion ejecutada: 3
Antes -> Ascensor:0,Posicion_persona:-1
Dejar

[(2, 0, 0), (1, 0), (3, 0, 0)]